In [7]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

# Building the molecular Hamiltonian

We first define the molecular structure and get the molecular Hamiltonian in terms of the tensor product of Pauli matrices. For the molecular hamiltonian of large molecules, we ignore the core orbitals to make the algorithm faster. The more active electrons and active orbitals we take, the slower and more accurate is the program. From the active electrons, we can construct the Hartree Fock state for our initialization.

In [8]:
bondlength = 1.128/0.529177 # Convert from Angstrom to atomic units

symbols = ["C", "O"]
geometry = np.array([[0.0, 0.0, -bondlength/2], [0.0, 0.0, bondlength/2]])
molecule = qchem.Molecule(symbols, geometry)
active_electrons = 6
active_orbitals = 6

H, qubits = qchem.molecular_hamiltonian(
    molecule,
    active_electrons=active_electrons,
    active_orbitals=active_orbitals
)
hf_state = qchem.hf_state(active_electrons, qubits)

We then find the total number of single and double excitations, and construct the corresponding operators necessary for our quantum circuit. 

In [9]:
singles, doubles = qchem.excitations(active_electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations

Total number of excitations = 117


In [10]:
def circuit_1(params, excitations):  # returns the expectation value of the Hamiltonian for our quantum circuit
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

# Adaptive step
Here we employ an adaptive process where we get rid of excitation gates who's coefficients have low impact on energy. We do that by calculating the gradient of the cost function with respect to the parameters associated with those excitation gates, calculated at a specific value of the parameters. We tried to randomize these values, but we have found that the value of the point at which the gradient is taken does not have any significant effect on the final result of the ground state and the spectral gap.

In [11]:
dev = qml.device("lightning.qubit", wires=qubits)
cost_fn_1 = qml.QNode(circuit_1, dev, interface="autograd")

circuit_gradient = qml.grad(cost_fn_1, argnum=0)

params = np.random.uniform(0.0, 2*np.pi, size = len(doubles)) # We chose the angles randomly
grads = circuit_gradient(params, excitations=doubles)

for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}") # List all the double excitation gates

doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
doubles_select # list the selected gates

Excitation : [0, 1, 6, 7], Gradient: 0.30390526913442883
Excitation : [0, 1, 6, 9], Gradient: 0.021278919065865765
Excitation : [0, 1, 6, 11], Gradient: 0.001064615559624902
Excitation : [0, 1, 7, 8], Gradient: -0.00036613890014146745
Excitation : [0, 1, 7, 10], Gradient: -0.00011812533092572794
Excitation : [0, 1, 8, 9], Gradient: 5.4669069638641494e-05
Excitation : [0, 1, 8, 11], Gradient: 1.1744313227009037e-05
Excitation : [0, 1, 9, 10], Gradient: 1.2184176656088524e-07
Excitation : [0, 1, 10, 11], Gradient: -1.836942997130597e-07
Excitation : [0, 2, 6, 8], Gradient: 8.766127316221375e-08
Excitation : [0, 2, 6, 10], Gradient: 6.801273453728817e-08
Excitation : [0, 2, 8, 10], Gradient: 1.7970751409560278e-07
Excitation : [0, 3, 6, 7], Gradient: -6.4337367242743136e-09
Excitation : [0, 3, 6, 9], Gradient: -4.508549315577041e-09
Excitation : [0, 3, 6, 11], Gradient: -2.636705836840709e-09
Excitation : [0, 3, 7, 8], Gradient: 8.866038348644783e-10
Excitation : [0, 3, 7, 10], Gradient: 

[[0, 1, 6, 7],
 [0, 1, 6, 9],
 [0, 1, 6, 11],
 [0, 1, 7, 8],
 [0, 1, 7, 10],
 [0, 1, 8, 9],
 [0, 1, 8, 11],
 [2, 3, 6, 7],
 [2, 3, 6, 9],
 [2, 3, 7, 8],
 [2, 3, 7, 10],
 [2, 3, 8, 9],
 [2, 3, 8, 11],
 [2, 3, 9, 10],
 [2, 3, 10, 11],
 [2, 4, 8, 10],
 [2, 5, 8, 9],
 [3, 5, 7, 9],
 [3, 5, 7, 11],
 [3, 5, 9, 11],
 [4, 5, 6, 7],
 [4, 5, 6, 9],
 [4, 5, 6, 11],
 [4, 5, 7, 8],
 [4, 5, 8, 9],
 [4, 5, 8, 11],
 [4, 5, 9, 10],
 [4, 5, 10, 11]]

In [12]:
# update circuit , use a few runs of the gradient descent optimizer to update the parameters
# associated with double excitation gates and then filter for single gates
opt = qml.GradientDescentOptimizer(stepsize=0.5)

params_doubles = np.zeros(len(doubles_select), requires_grad=True)

for n in range(20):
    params_doubles = opt.step(cost_fn_1, params_doubles, excitations=doubles_select)

def circuit_2(params, excitations, gates_select, params_select): 
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)

cost_fn_2 = qml.QNode(circuit_2, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn_2, argnum=0)
params = [0.0] * len(singles)

grads = circuit_gradient(params, excitations=singles, gates_select=doubles_select, params_select=params_doubles)

for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}") # lists all single gates

singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]
singles_select # lists selected single gates

Excitation : [0, 6], Gradient: 0.013600038023769082
Excitation : [0, 8], Gradient: 0.00018023564797461157
Excitation : [0, 10], Gradient: 0.0
Excitation : [1, 7], Gradient: -0.013600652297376208
Excitation : [1, 9], Gradient: -9.998897944237843e-05
Excitation : [1, 11], Gradient: 0.0
Excitation : [2, 6], Gradient: -8.739833982017579e-05
Excitation : [2, 8], Gradient: 0.015328442211633472
Excitation : [2, 10], Gradient: 0.0
Excitation : [3, 7], Gradient: 0.00021019762988084694
Excitation : [3, 9], Gradient: -0.01532780935888934
Excitation : [3, 11], Gradient: 0.0
Excitation : [4, 6], Gradient: 0.0
Excitation : [4, 8], Gradient: 0.0
Excitation : [4, 10], Gradient: -0.01430308111571473
Excitation : [5, 7], Gradient: 0.0
Excitation : [5, 9], Gradient: 0.0
Excitation : [5, 11], Gradient: 0.014303331437686962


[[0, 6],
 [0, 8],
 [1, 7],
 [1, 9],
 [2, 6],
 [2, 8],
 [3, 7],
 [3, 9],
 [4, 10],
 [5, 11]]

# Optimization for finding the ground state using VQE
We use the gradient descent optimizer in pennylane to optimize our cost function  and get the ground state.

In [13]:
opt = qml.GradientDescentOptimizer(stepsize=0.5)
max_iter = 1000
tolerance = 1e-5

cost_fn_3 = qml.QNode(circuit_1, dev, interface="autograd")
# Choosing initial value of params to be random to reduce chances of getting stuck in local minima
#params = np.random.uniform(0.0, 2*np.pi, size = len(doubles_select + singles_select), requires_grad=True)
params = np.zeros(len(doubles_select + singles_select), requires_grad=True)

gates_select = doubles_select + singles_select
prev_energy = 0

for n in range(max_iter):
    t1 = time.time()
    params, energy = opt.step_and_cost(cost_fn_3, params, excitations=gates_select)
    t2 = time.time()
    print("n = {:},  E = {:.8f} H, t = {:.2f} s".format(n, energy, t2 - t1))
    if abs(prev_energy - energy) < tolerance:
        break
    else:
        prev_energy = energy
ground_energy = energy
ground_params, excitations = params, gates_select
print(f"Ground state energy = {ground_energy} Ha")

n = 0,  E = -111.22455875 H, t = 0.29 s
n = 1,  E = -111.25813600 H, t = 0.20 s
n = 2,  E = -111.27373098 H, t = 0.20 s
n = 3,  E = -111.28150741 H, t = 0.30 s
n = 4,  E = -111.28557274 H, t = 0.20 s
n = 5,  E = -111.28778505 H, t = 0.30 s
n = 6,  E = -111.28903758 H, t = 0.20 s
n = 7,  E = -111.28977592 H, t = 0.30 s
n = 8,  E = -111.29022900 H, t = 0.20 s
n = 9,  E = -111.29051793 H, t = 0.20 s
n = 10,  E = -111.29070881 H, t = 0.30 s
n = 11,  E = -111.29083890 H, t = 0.19 s
n = 12,  E = -111.29092996 H, t = 0.30 s
n = 13,  E = -111.29099513 H, t = 0.20 s
n = 14,  E = -111.29104263 H, t = 0.21 s
n = 15,  E = -111.29107778 H, t = 0.30 s
n = 16,  E = -111.29110410 H, t = 0.20 s
n = 17,  E = -111.29112401 H, t = 0.30 s
n = 18,  E = -111.29113918 H, t = 0.20 s
n = 19,  E = -111.29115082 H, t = 0.20 s
n = 20,  E = -111.29115980 H, t = 0.30 s
Ground state energy = -111.29115980178311 Ha


# The first excited state using VQD
Now that we have the ground state, we can use it to find the first excited state. VQD tries to minimize the expectation value of the hamiltonian, but applies a penalty for any overlap with the ground state.

In [14]:
# Apply the ground state to the appropriate group of wires
def prepare_ground_state():
    qml.BasisState(hf_state, wires=range(1, qubits+1))

    for i, excitation in enumerate(excitations):
        #print([w+1 for w in excitation])
        if len(excitation) == 4:
            qml.DoubleExcitation(ground_params[i], wires=[w+1 for w in excitation])
        else:
            qml.SingleExcitation(ground_params[i], wires=[w+1 for w in excitation])

# Applies our ansatz to wires, with the option to move the whole circuit downwards
def ansatz(params, offset=0):
    qml.BasisState(hf_state, wires=range(offset, qubits+offset))

    for i, excitation in enumerate(excitations):
        #print([w+offset for w in excitation])
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=[w+offset for w in excitation])
        else:
            qml.SingleExcitation(params[i], wires=[w+offset for w in excitation])

# Calculate's the overlap between the ground state qubits and our current parameters using the swap test circuit
dev_large = qml.device('lightning.qubit', wires=2*qubits+1)
@qml.qnode(dev_large)
def swap_test(params):
    prepare_ground_state()
    ansatz(params, offset=qubits+1)

    qml.Barrier()  # added to better visualise the circuit
    qml.Hadamard(wires=0)
    for i in range(qubits):
        qml.CSWAP(wires=[0, 1 + i + qubits, 1 + i])
    qml.Hadamard(wires=0)
    return qml.expval(qml.Z(0))
    
beta = 0.2
def VQD_cost(params, excitations):
    return cost_fn_3(params, excitations) + beta * swap_test(params)

params = np.zeros(len(excitations), requires_grad=True)

In [15]:
print(qml.draw(swap_test)(params))

 0: ──────────────────────────────────────────────────────────────────────────────────────
 1: ─╭|Ψ⟩─╭G²(0.29)─╭G²(0.00)─╭G²(0.00)─╭G²(-0.00)─╭G²(0.00)─╭G²(0.02)─╭G²(0.00)──────────
 2: ─├|Ψ⟩─├G²(0.29)─├G²(0.00)─├G²(0.00)─├G²(-0.00)─├G²(0.00)─├G²(0.02)─├G²(0.00)──────────
 3: ─├|Ψ⟩─│─────────│─────────│─────────│──────────│─────────│─────────│─────────╭G²(0.02)
 4: ─├|Ψ⟩─│─────────│─────────│─────────│──────────│─────────│─────────│─────────├G²(0.02)
 5: ─├|Ψ⟩─│─────────│─────────│─────────│──────────│─────────│─────────│─────────│────────
 6: ─├|Ψ⟩─│─────────│─────────│─────────│──────────│─────────│─────────│─────────│────────
 7: ─├|Ψ⟩─├G²(0.29)─├G²(0.00)─├G²(0.00)─│──────────│─────────│─────────│─────────├G²(0.02)
 8: ─├|Ψ⟩─╰G²(0.29)─│─────────│─────────├G²(-0.00)─├G²(0.00)─│─────────│─────────╰G²(0.02)
 9: ─├|Ψ⟩───────────│─────────│─────────╰G²(-0.00)─│─────────├G²(0.02)─├G²(0.00)──────────
10: ─├|Ψ⟩───────────╰G²(0.00)─│────────────────────│─────────╰G²(0.02)─│──────────────────

In [16]:
# Minimize the VQD loss while checking for convergence
tol = 1e-4
diff = 1
params, lastEnergy = opt.step_and_cost(VQD_cost, params, excitations = excitations)
for n in range(10000):
    t1 = time.time()
    params, energy = opt.step_and_cost(VQD_cost, params, excitations=excitations)
    t2 = time.time()
    if(abs(lastEnergy - energy) <= tol):
        break
    else:
        lastEnergy = energy
    print(f't: {t2-t1}, cost: {energy}')

t: 64.93617653846741, cost: -111.06060667928692
t: 62.0795578956604, cost: -111.0731540241694
t: 62.15781307220459, cost: -111.08006368339792
t: 62.34342908859253, cost: -111.08405478859633
t: 63.29064154624939, cost: -111.08644520955141
t: 74.61982345581055, cost: -111.08792452225254
t: 63.388060331344604, cost: -111.08886985921849
t: 63.81554293632507, cost: -111.0894936264795
t: 63.67930746078491, cost: -111.08991835675893
t: 63.754223346710205, cost: -111.09021636849309
t: 63.136861085891724, cost: -111.09043135354341
t: 61.85248303413391, cost: -111.0905903621635
t: 63.293318033218384, cost: -111.09071057292265


In [17]:
# classical evaluation of the hamiltonian's eigenvalues
ham = H.sparse_matrix().toarray()
eigs = np.linalg.eigvalsh(ham)
actual = abs(eigs[0] - eigs[1])
print(actual)

0.25546694914990553


In [18]:
calculated = abs(ground_energy - energy)
print(calculated)
print(abs(actual - calculated)/abs(actual))

0.20035661915908065
0.21572391330546117


In [19]:
print(eigs)

[-111.33086183 -111.07539488 -111.07539488 ... -101.94404338 -100.29517962
  -99.34429395]
